In [4]:
import torch

# Humandoid MARL
from Humanoid_MARL import envs
from Humanoid_MARL.envs.base_env import GymWrapper, VectorGymWrapper
from Humanoid_MARL.utils.visual import save_video, save_rgb_image
from Humanoid_MARL.utils.torch_utils import save_models, load_models
from Humanoid_MARL.agent.ppo.train_torch import eval_unroll, get_agent_actions
from Humanoid_MARL.agent.ppo.agent import Agent
from Humanoid_MARL.utils.logger import WandbLogger
from Humanoid_MARL.agent.ppo.train_torch import train
import matplotlib.pyplot as plt
from datetime import datetime
import numpy as np
import os
# from Humanoid_MARL.envs.torch_wrapper import TorchWrapper
from brax.envs.wrappers import torch as torch_wrapper
from IPython.display import HTML, clear_output
from brax.io import html
import jax
from Humanoid_MARL import envs

In [5]:
env_name = "humanoid"
project_name = f"MARL_ppo_{env_name}_debug"

config = {
            'num_timesteps': 50_000_000,
            'eval_reward_limit' : 15_000,
            'eval_frequency': 10,
            'episode_length': 1000,
            'unroll_length': 10,
            'num_minibatches': 32,
            'num_update_epochs': 8,
            'discounting': 0.97,
            'learning_rate': 3e-4,
            'entropy_cost': 2e-3,
            'num_envs': 2048,
            'batch_size': 512,
            'env_name': env_name,
            'device' : 'cuda',
            'debug' : False,
            'device_idx' : 0
        }

In [6]:
logger = WandbLogger(project_name, config=config)
config['logger'] = logger
    # ================ Progress Function ================
xdata = []
ydata = []
eval_sps = []
train_sps = []
times = [datetime.now()]

def progress(num_steps, metrics, path="../data/ppo", name="ppo_training_plot.png"):
    times.append(datetime.now())
    xdata.append(num_steps)
    ydata.append(metrics["eval/episode_reward"].cpu())
    eval_sps.append(metrics["speed/eval_sps"])
    train_sps.append(metrics["speed/sps"])
    plt.xlim([0, config['num_timesteps']])
    plt.ylim([0, 10_000])
    plt.xlabel("# environment steps")
    plt.ylabel("reward per episode")
    plt.plot(xdata, ydata)
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    timestamped_name = f"{timestamp}_{name}"
    PLT_SAVE_PATH = os.path.join(path, timestamped_name)
    plt.savefig(PLT_SAVE_PATH)
# ================ Progress Function ================
agents = train(**config, progress_fn=progress)
print(f"time to jit: {times[1] - times[0]}")
print(f"time to train: {times[-1] - times[1]}")
print(f"eval steps/sec: {np.mean(eval_sps)}")
print(f"train steps/sec: {np.mean(train_sps)}")



wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


eval/episode_reward,▁
losses/total_loss,▁
speed/eval_sps,▁
speed/sps,▁
eval/episode_reward,108.23025
losses/total_loss,0
speed/eval_sps,70380.37821
speed/sps,0


Current Epoch [0] / [30]
mini_loss [0] | 0.5923230648040771
mini_loss [1] | 0.5784776210784912
mini_loss [2] | 0.5694406628608704
mini_loss [3] | 0.5688638687133789
mini_loss [4] | 0.5676000714302063
mini_loss [5] | 0.5638981461524963
mini_loss [6] | 0.5469768643379211
mini_loss [7] | 0.5549589395523071
mini_loss [8] | 0.5272847414016724
mini_loss [9] | 0.49853572249412537
mini_loss [10] | 0.5439041256904602
mini_loss [11] | 0.4990377128124237
mini_loss [12] | 0.5016807317733765
mini_loss [13] | 0.4909968674182892
mini_loss [14] | 0.47856682538986206
mini_loss [15] | 0.5039425492286682
mini_loss [16] | 0.49367374181747437
mini_loss [17] | 0.5005607604980469
mini_loss [18] | 0.4734359383583069
mini_loss [19] | 0.47806236147880554
mini_loss [20] | 0.4454229176044464
mini_loss [21] | 0.4447548985481262
mini_loss [22] | 0.45587265491485596
mini_loss [23] | 0.4338878095149994
mini_loss [24] | 0.37884509563446045
mini_loss [25] | 0.42766281962394714
mini_loss [26] | 0.44048693776130676
mini_

In [ ]:
print(f"time to jit: {times[1] - times[0]}")
print(f"time to train: {times[-1] - times[1]}")
print(f"eval steps/sec: {np.mean(eval_sps)}")
print(f"train steps/sec: {np.mean(train_sps)}")